In [1]:
from __future__ import annotations

import os

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch import optim
from tqdm import tqdm
from my_agents import A2C
import gymnasium as gym

In [2]:
""" Init """
# Parametri env
n_envs = 20 
n_updates = 2000
n_steps_per_update = 128

#  Parametri agente
gamma = 0.999
lam = 0.95
ent_coef = 0.01
actor_lr = 0.001    # Learning rate actor
critic_lr = 0.005   # Learning rate critic

# Genero vettore contente tutti gli env
envs = gym.vector.make("LunarLander-v2", num_envs=n_envs, max_episode_steps=600)
# envs = gym.make_vec("LunarLander-v2", num_envs=n_envs, max_episode_steps=600)

# Init osservazioni e azioni
obs_shape = envs.single_observation_space.shape[0]
action_shape = envs.single_action_space.n

device = torch.device("cpu")

# Init agente
agent = A2C(obs_shape, action_shape, device, critic_lr, actor_lr, n_envs)


C:\Users\paolo\miniconda3\envs\gymenv\Lib\site-packages\gymnasium\vector\__init__.py:53: UserWarning: WARN: `gymnasium.vector.make(...)` is deprecated and will be replaced by `gymnasium.make_vec(...)` in v1.0
  gym.logger.warn(


In [ ]:
#  Generazion wrapped env
envs_wrapper = gym.wrappers.RecordEpisodeStatistics(envs, deque_size=n_envs*n_updates)

critic_losses = []
actor_losses = []
entropies = []

for sample_phase in tqdm(range(n_updates)):
    
    # Reset liste che contengono l'esperienza di ogni episodio
    ep_value_preds      = torch.zeros(n_steps_per_update, n_envs, device=device)
    ep_rewards          = torch.zeros(n_steps_per_update, n_envs, device=device)
    ep_action_log_probs = torch.zeros(n_steps_per_update, n_envs, device=device)
    masks               = torch.zeros(n_steps_per_update, n_envs, device=device)

    if sample_phase == 0:
        states, info = envs_wrapper.reset(seed=42)

    # Eseguo n step su ogni env x ottenere dati
    for step in range(n_steps_per_update):

        #  L'agente seleziona un'azione in base allo stato corrente
        actions, action_log_probs, state_value_preds, entropy = agent.select_action(states)

        #  Eseguo l'azione selezionata nell'ambiente
        states, rewards, terminated, truncated, infos = envs_wrapper.step(actions.cpu().numpy())


        #  Salvo i dati dell'iterazione
        ep_value_preds[step] = torch.squeeze(state_value_preds)
        ep_rewards[step] = torch.tensor(rewards, device=device)
        ep_action_log_probs[step] = action_log_probs

        #  Aggiungo una mask che vale 1 se l'episiodio sta ancora runnando e vale 0 altrimenti
        masks[step] = torch.tensor([not term for term in terminated])

    #  Terminate le iterazioni calcolo le perdite per attore e critico
    critic_loss, actor_loss = agent.get_losses(
        ep_rewards, 
        ep_action_log_probs, 
        ep_value_preds, 
        entropy, 
        masks,
        gamma,
        lam, 
        ent_coef,
        device,
    )

    #  Update network attore e critico
    agent.update_parameters(critic_loss, actor_loss)

    #  Salvo perdite di attore e critico e entropia
    critic_losses.append(critic_loss.detach().cpu().numpy())    
    actor_losses.append(actor_loss.detach().cpu().numpy())
    entropies.append(entropy.detach().cpu().numpy())





  0%|▎                                                                                | 9/2000 [00:03<10:51,  3.06it/s]

In [ ]:
""" plot the results """

# %matplotlib inline

rolling_length = 20
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12, 5))
fig.suptitle(
    f"Training plots for {agent.__class__.__name__} in the LunarLander-v2 environment \n \
             (n_envs={n_envs}, n_steps_per_update={n_steps_per_update})"
)

# episode return
axs[0][0].set_title("Episode Returns")
episode_returns_moving_average = (
    np.convolve(
        np.array(envs_wrapper.return_queue).flatten(),
        np.ones(rolling_length),
        mode="valid",
    )
    / rolling_length
)
axs[0][0].plot(
    np.arange(len(episode_returns_moving_average)) / n_envs,
    episode_returns_moving_average,
)
axs[0][0].set_xlabel("Number of episodes")

# entropy
axs[1][0].set_title("Entropy")
entropy_moving_average = (
    np.convolve(np.array(entropies), np.ones(rolling_length), mode="valid")
    / rolling_length
)
axs[1][0].plot(entropy_moving_average)
axs[1][0].set_xlabel("Number of updates")


# critic loss
axs[0][1].set_title("Critic Loss")
critic_losses_moving_average = (
    np.convolve(
        np.array(critic_losses).flatten(), np.ones(rolling_length), mode="valid"
    )
    / rolling_length
)
axs[0][1].plot(critic_losses_moving_average)
axs[0][1].set_xlabel("Number of updates")


# actor loss
axs[1][1].set_title("Actor Loss")
actor_losses_moving_average = (
    np.convolve(np.array(actor_losses).flatten(), np.ones(rolling_length), mode="valid")
    / rolling_length
)
axs[1][1].plot(actor_losses_moving_average)
axs[1][1].set_xlabel("Number of updates")

plt.tight_layout()
plt.show()

In [ ]:
# Path di salvataggio
actor_weights_path = "weights/actor_weights.h5"
critic_weights_path = "weights/critc_weights.h5"

if not os.path.exists("weights"):
    os.mkdir("weights")

#  Salvataggio dati agente A2C
torch.save(agent.actor.state_dict(), actor_weights_path)
torch.save(agent.critic.state_dict(), critic_weights_path)